https://colab.research.google.com/drive/1P9iQHtUH5KUbTVtp8B4-AopZzEEPE0lw?usp=sharing#scrollTo=4YS7XI2bZTyz

https://github.com/aub-mind/arabert

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install optuna==2.3.0
!pip -q install transformers==4.2.1
!pip -q install farasapy
!pip -q install pyarabic
# !git clone https://huggingface.co/qarib/bert-base-qarib

     |████████████████████████████████| 258 kB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 164 kB 47.2 MB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 
     |████████████████████████████████| 75 kB 4.2 MB/s 
     |████████████████████████████████| 141 kB 49.6 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 111 kB 51.7 MB/s 
     |████████████████████████████████| 1.8 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 35.9 MB/s 
     |████████████████████████████████| 2.9 MB 36.4 MB/s 
     |████████████████████████████████| 126 kB 4.1 MB/s 


In [3]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.2 MB/s 


In [4]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

In [5]:
hard_map = { 0 : 'fake', 1 : 'real', }

df_train = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataDescr/PPV2train_news.csv", header=0,encoding='utf-8')
df_train.label= df_train.label.apply(lambda x: hard_map[x])

df_val = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataDescr/PPV2val_news.csv", header=0,encoding='utf-8')
df_val.label= df_val.label.apply(lambda x: hard_map[x])

label_list_HARD = ['fake', 'real']


# Trainer

In [6]:
# from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures


In [7]:
#https://huggingface.co/qarib/bert-base-qarib60_1970k
dataset_name = 'NEWS'
model_name = 'kuisailab/albert-base-arabic'
model_name = 'kuisailab/albert-base-arabic'
task_name = 'classification'
max_len = 414

In [8]:
# arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

# df_train.text = df_train.text.apply(lambda x:   arabert_prep.preprocess(x))
# df_val.text = df_val.text.apply(lambda x:   arabert_prep.preprocess(x))  

In [9]:
class BERTDataset():
    def __init__(self, text=None, target= None, model_name = None, max_len= None, label_map= None):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

# 

In [10]:
label_map = { v:index for index, v in enumerate(label_list_HARD) }
print(label_map)
train_dataset = BERTDataset(df_train.text.to_list(),df_train.label.to_list(),model_name, max_len,label_map)
val_dataset  = BERTDataset(df_val.text.to_list()  ,df_val.label.to_list()  ,model_name,  max_len, label_map)


{'fake': 0, 'real': 1}


Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/860k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

In [11]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [12]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds))
  print(confusion_matrix(p.label_ids,preds))

  # macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  # c_repoert= classification_report(p.label_ids,preds))

  return {
      'macro_f1' : macro_f1,
      # 'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc,
      # 'classification_report' :c_repoert
  }

# Regular Training

In [13]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = True
# training_args.per_device_train_batch_size =16
# training_args.per_device_eval_batch_size = 16

training_args.per_device_train_batch_size = 16  
training_args.per_device_eval_batch_size = 16


training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8

steps_per_epoch = (len(df_train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.load_best_model_at_end=True

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

160
1280


In [14]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
) 

Downloading:   0%|          | 0.00/47.3M [00:00<?, ?B/s]

Some weights of the model checkpoint at kuisailab/albert-base-arabic were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at kuisailab/albert-base-arabic and are newly initialized: ['classifier.weight'

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
0,No log,0.218257,0.927477,0.956660,0.905894,0.946178,113.325000,11.313000
1,No log,0.171108,0.940133,0.928605,0.954033,0.951638,112.517800,11.394000
2,No log,0.207382,0.951195,0.951614,0.950777,0.961778,112.565700,11.389000
3,0.161500,0.222216,0.949776,0.954164,0.945619,0.960998,112.875600,11.358000
4,0.161500,0.274192,0.945660,0.960722,0.932918,0.958658,112.293400,11.417000
5,0.161500,0.261315,0.951508,0.958781,0.944833,0.962559,113.298700,11.315000
6,0.011400,0.271388,0.948062,0.949756,0.946404,0.959438,113.403800,11.305000
7,0.011400,0.271943,0.948062,0.949756,0.946404,0.959438,113.049400,11.340000


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:911: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,


              precision    recall  f1-score   support

           0       0.98      0.82      0.89       343
           1       0.94      0.99      0.96       939

    accuracy                           0.95      1282
   macro avg       0.96      0.91      0.93      1282
weighted avg       0.95      0.95      0.94      1282

[[281  62]
 [  7 932]]
              precision    recall  f1-score   support

           0       0.87      0.96      0.91       343
           1       0.98      0.95      0.97       939

    accuracy                           0.95      1282
   macro avg       0.93      0.95      0.94      1282
weighted avg       0.95      0.95      0.95      1282

[[329  14]
 [ 48 891]]
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       343
           1       0.97      0.97      0.97       939

    accuracy                           0.96      1282
   macro avg       0.95      0.95      0.95      1282
weighted avg       0.96     

TrainOutput(global_step=1280, training_loss=0.06757261849415955, metrics={'train_runtime': 10874.587, 'train_samples_per_second': 0.118, 'total_flos': 1190523978456768, 'epoch': 8.0})

In [16]:
trainer.evaluate(val_dataset)

              precision    recall  f1-score   support

           0       0.93      0.92      0.92       343
           1       0.97      0.97      0.97       939

    accuracy                           0.96      1282
   macro avg       0.95      0.95      0.95      1282
weighted avg       0.96      0.96      0.96      1282

[[315  28]
 [ 24 915]]


{'epoch': 8.0,
 'eval_accuracy': 0.9594383775351014,
 'eval_loss': 0.27194327116012573,
 'eval_macro_f1': 0.9480617425151381,
 'eval_macro_precision': 0.9497555344926285,
 'eval_macro_recall': 0.9464041207537328,
 'eval_runtime': 113.0838,
 'eval_samples_per_second': 11.337}

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [18]:
trainer._save("/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/QaRiB")
tokenizer.save_pretrained("/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/QaRiB")

('/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/QaRiB/tokenizer_config.json',
 '/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/QaRiB/special_tokens_map.json',
 '/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/QaRiB/spiece.model',
 '/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/QaRiB/added_tokens.json')

# Test

In [19]:
df_test = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataDescr/PPV2test_news.csv")
print(df_test.label.value_counts())


1    1203
0     400
Name: label, dtype: int64


In [20]:
df_test.label= df_test.label.apply(lambda x: hard_map[x])

In [21]:
test_dataset = BERTDataset(df_test.text.to_list(),df_test.label.to_list(),model_name, max_len,label_map)

In [22]:
trainer.evaluate(test_dataset)

              precision    recall  f1-score   support

           0       0.92      0.92      0.92       400
           1       0.97      0.97      0.97      1203

    accuracy                           0.96      1603
   macro avg       0.95      0.95      0.95      1603
weighted avg       0.96      0.96      0.96      1603

[[ 369   31]
 [  33 1170]]


{'epoch': 8.0,
 'eval_accuracy': 0.9600748596381784,
 'eval_loss': 0.25900202989578247,
 'eval_macro_f1': 0.9467886025369189,
 'eval_macro_precision': 0.9460493121403806,
 'eval_macro_recall': 0.9475342892768079,
 'eval_runtime': 141.9915,
 'eval_samples_per_second': 11.289}

# Predict

In [23]:
class Dataset():
    def __init__(self, text=None, target= None, model_name = None, max_len= None, label_map= None):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask)

In [24]:

title_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/title_example.csv")
title_example.shape

count=0
for title in title_example.text:
   test_dataset = Dataset([title],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(test_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1

    
print("Real : "+str(count))

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
Real : 3


In [25]:
text_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/text_example.csv")
text_example.shape
count=0

for text in text_example.text:
   text_dataset = Dataset([text],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(text_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1
    
print("Real : "+str(count))

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
Real : 0


In [26]:
# # Tokenize test data
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512) 
# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized) 
# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2) 
# # Define test trainer
# test_trainer = Trainer(model) 
# # Make prediction
# raw_pred, _, _ = test_trainer.predict(test_dataset) 
# # Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)

In [27]:
# print("===> Predicting...")
# trainer1 = Trainer(
#     model= model_init(),
#     args=training_args,
#     eval_dataset=test_dataset
# )
        
# results = {}
# result = trainer1.predict(test_dataset)

In [28]:
#  ----- 3. Predict -----#
# # Load test data
# test_data = pd.("drive/My Drive/Arabic_FakeNews/Model/Data/test_news.csv", header=0,encoding='utf-8')
# X_test = list(test_data["text"])
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized)

# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# # Define test trainer
# test_trainer = Trainer(model)

# # Make prediction
# raw_pred, _, _ = test_trainer.predict(test_dataset)

# # Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)